In [1]:
# Script to clean uspatentcitation.tsv

# Jan 20th, 2020
# There are citation_ids larger than 7 characters and smaller than 4
# Larger are usually applications, smallers tend to be errors
# I am keeping than so calculations on forward citations are accurate
# When matching by citation_id, it must be previously filtered

# as of Jan 9th, 2020, there are entries to be evaluated
# for now, error_bad_lines=False skips those entries


In [2]:
import sys
sys.path.append('/home/rkogeyam/scripts/')
from sampler import sampler

import pandas as pd
import numpy as np
import re

In [3]:
# patent.csv
# id:       patent this record corresponds to 
# type:     category of patent. Usually "Design", "reissue", etc.
# number:   patent number
# country:  country in which patent was granted (always US)
# date:     date when patent was granted
# abstract: abstract text of patent
# title:    title of patent
# kind:     WIPO document kind codes (http://www.uspto.gov/learning-and-resources/support-centers/electronic-business-center/kind-codes-included-uspto-patent)
# num_claims:number of claims
# filename: name of the raw data file where patent information is parsed from

In [4]:
src= r'/home/rkogeyam/PATENT_CITATION/data/patent.csv'
dst= '/home/rkogeyam/PATENT_CITATION/data/cleanpatent.csv'

wipo_first_class= '/home/rkogeyam/PATENT_CITATION/data/wipo_first_class.csv'

In [5]:
cols=['id', 'num_claims', 'date', 'type', 'kind']

In [6]:
df = pd.read_csv(src, sep='\t', usecols=cols, error_bad_lines=False, dtype=object)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6488267 entries, 0 to 6488266
Data columns (total 5 columns):
id            object
type          object
date          object
kind          object
num_claims    object
dtypes: object(5)
memory usage: 247.5+ MB


In [8]:
df.dtypes

id            object
type          object
date          object
kind          object
num_claims    object
dtype: object

In [9]:
%%time
# Keep this for reference!
# As of Dec 31st, 2019, I compared the clean to the raw version of citation and patent ids

# stripping non-desired characters but keeping the originals for later check - only three changes in citation_id

cleaning_patent=lambda x:re.sub('([^a-zA-Z0-9]+)', "", x)
df['id']=df['id'].apply(cleaning_patent)

CPU times: user 17.3 s, sys: 1 ms, total: 17.3 s
Wall time: 17.3 s


In [10]:
df.date.replace({'-00':'-01'}, regex=True, inplace=True)

In [11]:
df.id.str.len().value_counts()

7      6480301
6         5995
8         1966
433          1
136          1
56           1
52           1
47           1
Name: id, dtype: int64

In [12]:
df[
df['id'].apply(lambda x: len(x)<4)
  ]

,id,type,date,kind,num_claims


In [13]:
df[
df['id'].apply(lambda x: len(x)>13)
  ]

,id,type,date,kind,num_claims
2640192,eceptionunitsSequentiallyduringdatatransmission,NaN,NaN,NaN,NaN
2640193,eceptionwhetherornotatransmissionsignalhasbeen...,NaN,NaN,NaN,NaN
2640194,eceptionunitreceivingsaidsignalisusedfordatatr...,NaN,NaN,NaN,NaN
2640195,eceptionviaopticalcommunicationswithanexternal...,Device provided with an optical communications...,pg030624.zip,NaN,NaN
3724933,nendofmessageindicatorandusesthevarioustestres...,Unsolicited message intercepting communication...,ipg100302.xml,NaN,NaN


In [14]:
df.to_csv(dst)

In [15]:
# df =df.merge(pd.read_csv(wipo_first_class, sep='\t', usecols=cols, error_bad_lines=False, index_col=0, dtype=object))